# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
!pip install pandas

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import song_table_insert, artist_table_insert, time_table_insert, user_table_insert, song_select, songplay_table_insert

/Users/marioreyes/.virtualenvs/ProjectUd/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files("data/song_data")

In [6]:
filepath = song_files[-1]

In [7]:
df = pd.read_json(filepath, typ='series')
df

num_songs                            1
artist_id           ARGCY1Y1187B9A4FA5
artist_latitude                36.1678
artist_longitude              -86.7784
artist_location         Nashville, TN.
artist_name                   Gloriana
song_id             SOQOTLQ12AB01868D0
title               Clementina Santafè
duration                       153.338
year                                 0
dtype: object

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = df.loc[['song_id', 'title', 'artist_id', 'year', 'duration']].T.values.tolist()
song_data

['SOQOTLQ12AB01868D0',
 'Clementina Santafè',
 'ARGCY1Y1187B9A4FA5',
 0,
 153.33832]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
artist_data = df.loc[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].T.values.tolist()
artist_data

['ARGCY1Y1187B9A4FA5', 'Gloriana', 'Nashville, TN.', 36.16778, -86.77836]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [12]:
log_files = get_files("data/log_data")

In [13]:
filepath = log_files[-1]

In [14]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Future Sound of London,Logged In,Ryan,M,0,Smith,405.28934,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,873,Papua New Guinea,200,1543020300796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
1,Harmonia,Logged In,Ryan,M,1,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,873,Sehr kosmisch,200,1543020705796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
2,None,Logged Out,None,None,0,None,NaN,free,None,GET,Home,NaN,86,None,200,1543026352796,None,
3,None,Logged Out,None,None,1,None,NaN,free,None,GET,Home,NaN,86,None,200,1543026402796,None,
4,None,Logged Out,None,None,2,None,NaN,free,None,GET,Help,NaN,86,None,200,1543026454796,None,


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [15]:
df = df[df['page'] == "NextSong"]
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Future Sound of London,Logged In,Ryan,M,0,Smith,405.28934,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,873,Papua New Guinea,200,1543020300796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
1,Harmonia,Logged In,Ryan,M,1,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,873,Sehr kosmisch,200,1543020705796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
9,My Chemical Romance,Logged In,Dustin,M,7,Lee,311.11791,free,"Myrtle Beach-Conway-North Myrtle Beach, SC-NC",PUT,NextSong,1.540853e+12,86,Welcome To The Black Parade (Album Version),200,1543026602796,Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7....,87
10,Disturbing Tha Peace / Ludacris / Mystikal / I-20,Logged In,Mohammad,M,0,Rodriguez,276.42730,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,888,Move Bitch,200,1543027714796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
11,Carlos PaiÃÂ£o,Logged In,Mohammad,M,1,Rodriguez,185.99138,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,888,Playback,200,1543027990796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88


In [16]:
t = df['ts'].apply(pd.Timestamp, **{'unit':'ms'}).sort_values()
t.head()

0    2018-11-24 00:45:00.796
1    2018-11-24 00:51:45.796
9    2018-11-24 02:30:02.796
10   2018-11-24 02:48:34.796
11   2018-11-24 02:53:10.796
Name: ts, dtype: datetime64[ns]

In [18]:
time_data = ([tsi.timestamp(), tsi.hour, tsi.day, tsi.weekofyear, tsi.month, tsi.year, tsi.dayofweek] for tsi in t)
column_labels = ('timestamp', 'hour', 'day', 'weekofyear', 'month', 'year', 'weekday')

In [19]:
time_df = pd.DataFrame(data=time_data, columns=column_labels)
time_df

,timestamp,hour,day,weekofyear,month,year,weekday
0,1.543020e+09,0,24,47,11,2018,5
1,1.543021e+09,0,24,47,11,2018,5
2,1.543027e+09,2,24,47,11,2018,5
3,1.543028e+09,2,24,47,11,2018,5
4,1.543028e+09,2,24,47,11,2018,5
...,...,...,...,...,...,...,...
309,1.543092e+09,20,24,47,11,2018,5
310,1.543093e+09,20,24,47,11,2018,5
311,1.543098e+09,22,24,47,11,2018,5
312,1.543098e+09,22,24,47,11,2018,5


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [20]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [21]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.